In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from joblib import Parallel, delayed
from tqdm.notebook import tqdm

from utils import do_cv_knn

# Support Vector Machines

Nesta aula vamos usar a SVM (*Support Vector Machine*) como algoritmo de classificação e comparar seu desempenho com o KNN. O objetivo deste caderno em relação a SVM é mostrar como usar a SVM como um classificador "caixa-preta". O treinamento da SVM é um procedimento computacionalmente complexo, que está fora do escopo desta disciplina.

Dadas instâncias com $n$ atributos, a SVM encontra um classificador linear de margem máxima suave que separa 2 classes em uma dimensionalidade $m$, tal que $m>n$. A projeção dos dados do espaço $n$-dimensional para um espaço $m$ dimensional onde acontece por meio de transformações não-lineares descritas por *kernels*. A idéia é que esta projeção torne um problema de classificação não-linear em um problema linearmente separável por um hiperplano com $m-1$ dimensões. A otimização do melhor hiperplano é feito com base nos pontos (instâncias) que determinam a margem suave. Estes pontos são conhecidos como **vetores de suporte**, do inglês *support vectors*.

Para ter uma idéia mais intuitiva sobre como a SVM funciona, consulte o material a seguir:

* [Support Vector Machines Part 1 (of 3): Main Ideas!!!](https://www.youtube.com/watch?v=efR1C6CvhmE)
* [Support Vector Machines Part 2: The Polynomial Kernel (Part 2 of 3)](https://www.youtube.com/watch?v=Toet3EiSFcM)
* [Support Vector Machines Part 3: The Radial (RBF) Kernel (Part 3 of 3)](https://www.youtube.com/watch?v=Qc5IyLW_hns)

## Base de Dados

Hoje vamos usar a base de dados *heart*. Esta base de dados contém informações clínicas e resultados de exames laboratoriais de pacientes cardíacos. O atributo de saída indica se o paciente morreu devido a algum problema cardíaco.

In [2]:
df = pd.read_csv('heart.csv')
df

,AGE_50,MD_50,SBP_50,DBP_50,HT_50,WT_50,CHOL_50,SES,CL_STATUS,MD_62,SBP_62,DBP_62,CHOL_62,WT_62,IHD_DX,DEATH
0,42,1,110,65,64,147,291,2,8,4,120,78,271,146,2,1
1,53,1,130,72,69,167,278,1,6,2,122,68,250,165,9,1
2,53,2,120,90,70,222,342,4,8,1,132,90,304,223,2,1
3,48,4,120,80,72,229,239,4,8,2,118,68,209,227,3,1
4,53,3,118,74,66,134,243,3,8,5,118,56,261,138,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,50,1,115,80,66,148,300,2,8,1,115,65,273,152,0,0
196,23,1,110,70,69,137,120,3,8,2,112,76,198,153,0,0
197,20,3,130,80,66,150,210,5,0,1,130,85,274,158,0,0
198,46,3,140,84,66,138,130,4,6,2,148,88,160,157,0,0


In [3]:
#verificando se há dados faltantes
df.isnull().sum().sum()

0

In [4]:
#Atributo de saída
y = df['DEATH'].values.ravel()

#Atributos de entrada
X = df.drop('DEATH', axis=1)

## KNN

In [5]:
#Avaliar o desempenho do KNN nesta base de dados usando validação cruzada. 
#Para cada fold, realiza uma busca exaustiva para escolher o melhor $k$.
accs_knn = do_cv_knn(X.values, y, 10, range(1, 20, 2))

Folds avaliados:   0%|          | 0/10 [00:00<?, ?it/s]

Está na hora de criar uma função para calcular as estatísticas das acurácias obtidas na validação cruzada. Também já vamos implementar uma função para imprimir estas estatísticas de forma adequada.

In [6]:
def calcular_estatisticas(resultados):
    return np.mean(resultados), np.std(resultados), np.min(resultados), np.max(resultados)

def imprimir_estatisticas(resultados):
    media, desvio, mini, maxi = calcular_estatisticas(resultados)
    print("Resultados: %.2f +- %.2f, min: %.2f, max: %.2f" % (media, desvio, mini, maxi))

In [7]:
imprimir_estatisticas(accs_knn)

Resultados: 0.69 +- 0.06, min: 0.60, max: 0.80


## SVM

A biblioteca *sklearn* utiliza uma implementação famosa da SVM chamada de libSVM. Esta implementação está no módulo ``sklearn.svm``. Neste módulo, a classe ``SVC`` implementa a SVM configurável, que já vem com vários *kernels* implementados. Na aula de hoje vamos usar o kernel *rbf* (*radial basis function*), que é comumente utilizado. Outros kernels bastante usados são os kernels *linear* e *poly*.

Dependendo do *kernel* escolhido, há diferentes hiperparâmetros que precisam ser otimizados.

O kernel *rbf* possui um parâmetro $\gamma$ (gamma) que controla a influência da distância entre os pontos no cálculo da margem suave. Este parâmetro deve ser otimizado para obter os melhores resultados. É usual fazer uma busca exaustiva usando validação cruzada para otimizar este parâmetro. Os valores comumente avaliados são: $1/(\text{n_atributos} * \text{var}(X) )$ (``scale`` no sklearn), $1/\text{n_atributos}$ (``auto`` no sklearn), $2\times10^{-2}$, $2\times10^{-3}$, $2\times10^{-4}$ e $2\times10^{-5}$.

Outro parâmetro importante é a largura da margem suave, comumente especificado como $C$. É usual fazer uma busca exaustiva usando validação cruzada para otimizar este parâmetro. Os valores comumente avaliados são: $1$, $10$, $100$, $1000$ e $10000$.

A função ``selecionar_melhor_svm`` faz uma busca exaustiva pela melhor combinação de $C$ e $\gamma$ usando um conjunto de validação. O procedimento pode ser facilmente alterado para usar validação cruzada para fazer esta busca (usando ``GridSearchCV``).


In [8]:
from sklearn.svm import SVC
import itertools

In [9]:
#Cs e gammas são listas com os valores a serem avaliados para os respectivos parâmetros.
def selecionar_melhor_svm(Cs, gammas, X_treino : np.ndarray, X_val : np.ndarray, 
                          y_treino : np.ndarray, y_val : np.ndarray, n_jobs=4):
    
    def treinar_svm(C, gamma, X_treino, X_val, y_treino, y_val):
        svm = SVC(C=C, gamma=gamma)
        svm.fit(X_treino, y_treino)
        pred = svm.predict(X_val)
        return accuracy_score(y_val, pred)
    
    #gera todas as combinações de parametros C e gamma, de acordo com as listas de valores recebidas por parametro.
    #Na prática faz o produto cartesiano entre Cs e gammas.
    combinacoes_parametros = list(itertools.product(Cs, gammas))
    
    #Treinar modelos com todas as combinações de C e gamma
    acuracias_val = Parallel(n_jobs=n_jobs)(delayed(treinar_svm)
                                       (c, g, X_treino, X_val, y_treino, y_val) for c, g in combinacoes_parametros)       
    
    melhor_val = max(acuracias_val)
    #Encontrar a combinação que levou ao melhor resultado no conjunto de validação
    melhor_comb = combinacoes_parametros[np.argmax(acuracias_val)]   
    melhor_c = melhor_comb[0]
    melhor_gamma = melhor_comb[1]
    
    #Treinar uma SVM com todos os dados de treino e validação usando a melhor combinação de C e gamma.
    svm = SVC(C=melhor_c, gamma=melhor_gamma)
    svm.fit(np.vstack((X_treino, X_val)), [*y_treino, *y_val])

    return svm, melhor_comb, melhor_val

#Implementa a validação cruzada para avaliar o desempenho da SVM na base de dados com as instâncias X e as saídas y.
#cv_splits indica o número de partições que devem ser criadas.
#Cs é a lista com os valores C que devem ser avaliados na busca exaustiva de parametros para a SVM.
#gammas s é a lista com os valores gamma que devem ser avaliados na busca exaustiva de parametros para a SVM.
def do_cv_svm(X, y, cv_splits, Cs=[1], gammas=['scale']):

    skf = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=1)

    acuracias = []
    
    pgb = tqdm(total=cv_splits, desc='Folds avaliados')
    
    for treino_idx, teste_idx in skf.split(X, y):

        X_treino = X[treino_idx]
        y_treino = y[treino_idx]

        X_teste = X[teste_idx]
        y_teste = y[teste_idx]

        X_treino, X_val, y_treino, y_val = train_test_split(X_treino, y_treino, stratify=y_treino, test_size=0.2, random_state=1)

        ss = StandardScaler()
        ss.fit(X_treino)
        X_treino = ss.transform(X_treino)
        X_teste = ss.transform(X_teste)
        X_val = ss.transform(X_val)

        svm, _, _ = selecionar_melhor_svm(Cs, gammas, X_treino, X_val, y_treino, y_val)
        pred = svm.predict(X_teste)

        acuracias.append(accuracy_score(y_teste, pred))
        
        pgb.update(1)
        
    pgb.close()
    
    return acuracias

In [10]:
accs_svm = do_cv_svm(X.values, y, 10, Cs=[1, 10, 100, 1000], gammas=['scale', 'auto', 2e-2, 2e-3, 2e-4])

Folds avaliados:   0%|          | 0/10 [00:00<?, ?it/s]

In [11]:
imprimir_estatisticas(accs_knn)

Resultados: 0.69 +- 0.06, min: 0.60, max: 0.80


In [12]:
imprimir_estatisticas(accs_svm)

Resultados: 0.71 +- 0.05, min: 0.65, max: 0.80


# Teste de Hipótese Nula pelo Teste t de Student

A acurácia com KNN foi $0.69\pm0.06$, enquanto a acurácia com a SVM foi de $0.71\pm0.05$. Considerando apenas a diferença na média, o resultado com a SVM parece ter sido superior ao resultado com KNN. Entretanto, note que os desvios-padrão dos dois resultados são relativamente altos. Por exemplo, podemos esperar que, para dados desconhecidos, a acurácia do KNN seja entre $0.63$ e $0.75$. Já para a SVM esperamos que a acurácia seja entre $0.66$ e $0.76$. Veja que há uma sobreposição grande de acurácias prováveis entre KNN e SVM. 

Assim, a pergunta central aqui é: a diferença entre as médias dos resultados obtidos pelo KNN e pela SVM é **estatisticamente significativa?** Para responder esta pergunta, existem os testes de hipótese estatísticos. Na terminologia estatística, quando queremos verificar se duas distribuições estatísticas (por exemplo a distribuição gaussiana que estamos assumindo ao representar os testes usando a média e o desvio-padrão) são diferentes, perguntamos se é possível rejeitar a **hipótese nula**. A hipótese nula é a hipótese que diz que não há diferença significativa entre as duas distribuições.

O teste t de Student pode ser usado para verificar se é possível rejeitar a hipótese nula quando estamos comparando a diferença na média de duas distribuições estatísticas. O teste é realizado em duas etapas:

1. Calcular o p-valor ($p$) da diferença entre as médias usando a estatística t;
2. Se $p \leq \alpha$, podemos rejeitar a hipótese nula. Ao rejeitar a hipótese nula estamos dizendo que a diferença entre as médias é estatisticamente significativa.
3. Caso o $p > \alpha$, não é possível rejeitar a hipótese nula. Desta forma, não é possível afirmar que a diferença entre as médias é estatisticamente significativa.

Como interpretar o valor de $p$? $p$ é um número entre 0 e 1 que indica a probabilidade que a diferença nas médias seja resultado de um efeito aleatório, ou seja, algo não-relacionado com o que está sendo comparado. Por exemplo, se ao comparar o desempenho os dois classificadores acima obtermos $p=0.1$, isto indicaria que há 10% de chance que a diferença nas médias obtidas com KNN e SVM seja oriunda do particionamento utilizado para o treino e teste. Ou seja, é mais provável que as diferenças obtidas sejam oriundas das diferenças no funcionamento dos algoritmos KNN e SVM.

Em outras palavras, quanto mais próximo $p$ estiver de 0, mais confiantes estamos que a diferença na média é significativa. Resta a pergunta: como escolher o valor para $\alpha$?

Um valor comumente utilizado para comparar modelos estatísticos na área de aprendizagem de máquina (e muitas outras áreas da ciência) é $\alpha=0.05$. Este valor é normalmente interpretado como o nível de confiança mínimo que aceitamos para rejeitar a hipótese nula, e, consequentemente, concluir que a diferença nos resultados é significativa.

A função ``ttest_ind_from_stats`` do módulo ``scipy.stats`` calcula a estatística t e o p-valor ($p$) a partir da média e do desvio-padrão das distribuições gaussianas que queremos comparar.


In [13]:
from scipy.stats import ttest_ind_from_stats

In [14]:
#Primeiramente calculamos a média e o desvio padrão dos resultados
media_knn, std_knn, _, _ = calcular_estatisticas(accs_knn)
media_svm, std_svm, _, _ = calcular_estatisticas(accs_svm)

#calcular o pvalor usando o teste t de Student para duas amostras independentes
_, pvalor = ttest_ind_from_stats(media_knn, std_knn, len(accs_knn), media_svm, std_svm, len(accs_svm))

In [15]:
pvalor

0.6866732489128373

In [16]:
pvalor<=0.05

False

Note que o p-valor obtido é maior que $0.05$. Portanto, não é possível rejeitar a hipótese nula. Desta forma, não há como afirmar que a diferença entre as médias dos resultados é significativa. Em outras palavras, *não é possível afirmar* que a SVM teve desempenho diferente do KNN nos testes que fizemos com esta base de dados.

Podemos empacotar este teste de hipótese em uma função:

In [17]:
def rejeitar_hip_nula(media_amostral1, desvio_padrao_amostral1, n1, media_amostral2, desvio_padrao_amostral2, n2, alpha=0.05):
    _, pvalor = ttest_ind_from_stats(media_amostral1, desvio_padrao_amostral1, n1, media_amostral2, desvio_padrao_amostral2, n2)
    return pvalor <= alpha

In [20]:
rejeitar_hip_nula(media_knn, std_knn, len(accs_knn), media_svm, std_svm, len(accs_svm))

False